<a href="https://colab.research.google.com/github/garciawitulski/Econometria/blob/main/IV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# install the package
!pip install econtools
import econtools.metrics as mt
from sklearn import datasets, linear_model, metrics

!pip install linearmodels
from linearmodels.iv import IV2SLS



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 536 kB 4.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 5.0 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.1.5 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.4.0 which is incompatible.
spacy 3.4.2 requires typing-extensions<4.2.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.4.0 which 

In [2]:
url = 'https://raw.githubusercontent.com/garciawitulski/Econometria/main/CigarettesSW'

df = pd.read_csv(url)

In [3]:
df.head(10)

,Unnamed: 0,state,year,cpi,population,packs,income,tax,price,taxs
0,1,AL,1985,1.076,3973000.0,116.486282,46014968,32.500004,102.181671,33.348335
1,2,AR,1985,1.076,2327000.0,128.534592,26210736,37.000000,101.474998,37.000000
2,3,AZ,1985,1.076,3184000.0,104.522614,43956936,31.000000,108.578751,36.170418
3,4,CA,1985,1.076,26444000.0,100.363037,447102816,26.000000,107.837341,32.104000
4,5,CO,1985,1.076,3209000.0,112.963539,49466672,31.000000,94.266663,31.000000
5,6,CT,1985,1.076,3201000.0,109.278351,60063368,42.000000,128.024994,51.483334
6,7,DE,1985,1.076,618000.0,143.851135,9927301,30.000000,102.491661,30.000000
7,8,FL,1985,1.076,11352000.0,122.181122,166919248,37.000000,115.290001,42.490002
8,9,GA,1985,1.076,5963000.0,127.234619,78364336,28.000000,97.025169,28.841833
9,10,IA,1985,1.076,2830000.0,113.745583,37902896,34.000000,101.842003,37.917000


In [4]:
df['rprice'] = df['price'] / df['cpi']
df['lnpacks'] = np.log(df['packs'])
df['lnrprice'] = np.log(df['rprice'])
df['salestax'] = (df['taxs'] - df['tax']) / df['cpi']
df['rincome'] = df['income'] / df['population'] / df['cpi']
df['lnrincome'] = np.log(df['rincome'])
df['cigtax'] = df['tax'] / df['cpi']

In [5]:
c1995 = df[(df.year == 1995)]

c1995.head(10)

,Unnamed: 0,state,year,cpi,population,packs,income,tax,price,taxs,rprice,lnpacks,lnrprice,salestax,rincome,lnrincome,cigtax
48,49,AL,1995,1.524,4262731.0,101.085434,83903280,40.500004,158.371338,41.904671,103.918206,4.615966,4.643604,0.921697,12.915347,2.558416,26.574807
49,50,AR,1995,1.524,2480121.0,111.042969,45995496,55.500000,175.542511,63.859169,115.185380,4.709917,4.746543,5.485019,12.169073,2.498898,36.417325
50,51,AZ,1995,1.524,4306908.0,71.954170,88870496,65.333328,198.607498,74.790825,130.319887,4.276029,4.869992,6.205707,13.539638,2.605622,42.869640
51,52,CA,1995,1.524,31493524.0,56.859306,771470144,61.000000,210.504669,74.771332,138.126430,4.040580,4.928169,9.036307,16.073591,2.777178,40.026249
52,53,CO,1995,1.524,3738061.0,82.582924,92946544,44.000000,167.350006,44.000000,109.809720,4.413803,4.698749,0.000000,16.315557,2.792119,28.871392
53,54,CT,1995,1.524,3265293.0,79.472191,104315120,74.000000,218.280502,86.355499,143.228683,4.375407,4.964443,8.107283,20.962359,3.042728,48.556433
54,55,DE,1995,1.524,718265.0,124.466599,18237436,48.000000,165.600006,48.000000,108.661426,4.824037,4.688237,0.000000,16.660734,2.813055,31.496064
55,56,FL,1995,1.524,14185403.0,93.074554,333525344,57.900002,187.717178,68.525505,123.174007,4.533401,4.813598,6.972116,15.427737,2.736167,37.992129
56,57,GA,1995,1.524,7188538.0,97.474625,159800448,36.000000,156.573074,37.431416,102.738243,4.579592,4.632184,0.939249,14.586546,2.680100,23.622048
57,58,IA,1995,1.524,2840860.0,92.401596,60170928,60.000000,190.889999,69.089996,125.255911,4.526144,4.830359,5.964565,13.897988,2.631744,39.370081


In [6]:
cig_s0 = mt.reg(c1995, 'lnpacks', ['lnrprice'], addcons=True,vce_type="robust")

cig_s0


Dependent variable:	lnpacks
N:			48
R-squared:		0.4058
Estimation method:	OLS
VCE method:		Robust
          coeff    se      t   p>t CI_low CI_high
lnrprice -1.213 0.195 -6.234 0.000 -1.605  -0.821
_cons    10.339 0.935 11.060 0.000  8.457  12.221

In [7]:
cig_s1 = mt.reg(c1995, 'lnrprice', ['salestax'], addcons=True,vce_type="robust")

print(cig_s1)

c1995['lcigp_pred'] = cig_s1.yhat
c1995.head(10)

Dependent variable:	lnrprice
N:			48
R-squared:		0.4710
Estimation method:	OLS
VCE method:		Robust
         coeff    se       t   p>t CI_low CI_high
salestax 0.031 0.005   6.355 0.000  0.021   0.040
_cons    4.617 0.029 159.644 0.000  4.558   4.675



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Unnamed: 0,state,year,cpi,population,packs,income,tax,price,taxs,rprice,lnpacks,lnrprice,salestax,rincome,lnrincome,cigtax,lcigp_pred
48,49,AL,1995,1.524,4262731.0,101.085434,83903280,40.500004,158.371338,41.904671,103.918206,4.615966,4.643604,0.921697,12.915347,2.558416,26.574807,4.644869
49,50,AR,1995,1.524,2480121.0,111.042969,45995496,55.500000,175.542511,63.859169,115.185380,4.709917,4.746543,5.485019,12.169073,2.498898,36.417325,4.785095
50,51,AZ,1995,1.524,4306908.0,71.954170,88870496,65.333328,198.607498,74.790825,130.319887,4.276029,4.869992,6.205707,13.539638,2.605622,42.869640,4.807241
51,52,CA,1995,1.524,31493524.0,56.859306,771470144,61.000000,210.504669,74.771332,138.126430,4.040580,4.928169,9.036307,16.073591,2.777178,40.026249,4.894222
52,53,CO,1995,1.524,3738061.0,82.582924,92946544,44.000000,167.350006,44.000000,109.809720,4.413803,4.698749,0.000000,16.315557,2.792119,28.871392,4.616546
53,54,CT,1995,1.524,3265293.0,79.472191,104315120,74.000000,218.280502,86.355499,143.228683,4.375407,4.964443,8.107283,20.962359,3.042728,48.556433,4.865674
54,55,DE,1995,1.524,718265.0,124.466599,18237436,48.000000,165.600006,48.000000,108.661426,4.824037,4.688237,0.000000,16.660734,2.813055,31.496064,4.616546
55,56,FL,1995,1.524,14185403.0,93.074554,333525344,57.900002,187.717178,68.525505,123.174007,4.533401,4.813598,6.972116,15.427737,2.736167,37.992129,4.830791
56,57,GA,1995,1.524,7188538.0,97.474625,159800448,36.000000,156.573074,37.431416,102.738243,4.579592,4.632184,0.939249,14.586546,2.680100,23.622048,4.645408
57,58,IA,1995,1.524,2840860.0,92.401596,60170928,60.000000,190.889999,69.089996,125.255911,4.526144,4.830359,5.964565,13.897988,2.631744,39.370081,4.799831


In [8]:
cig_s2 = mt.reg(c1995, 'lnpacks', ['lcigp_pred'], addcons=True,vce_type="robust")
print(cig_s2)

from statsmodels.api import add_constant

c1995 = add_constant(c1995,has_constant='add')

c1995.head(10)

Dependent variable:	lnpacks
N:			48
R-squared:		0.1525
Estimation method:	OLS
VCE method:		Robust
            coeff    se      t   p>t CI_low CI_high
lcigp_pred -1.084 0.334 -3.247 0.002 -1.755  -0.412
_cons       9.720 1.597  6.086 0.000  6.505  12.935



/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,const,Unnamed: 0,state,year,cpi,population,packs,income,tax,price,taxs,rprice,lnpacks,lnrprice,salestax,rincome,lnrincome,cigtax,lcigp_pred
48,1.0,49,AL,1995,1.524,4262731.0,101.085434,83903280,40.500004,158.371338,41.904671,103.918206,4.615966,4.643604,0.921697,12.915347,2.558416,26.574807,4.644869
49,1.0,50,AR,1995,1.524,2480121.0,111.042969,45995496,55.500000,175.542511,63.859169,115.185380,4.709917,4.746543,5.485019,12.169073,2.498898,36.417325,4.785095
50,1.0,51,AZ,1995,1.524,4306908.0,71.954170,88870496,65.333328,198.607498,74.790825,130.319887,4.276029,4.869992,6.205707,13.539638,2.605622,42.869640,4.807241
51,1.0,52,CA,1995,1.524,31493524.0,56.859306,771470144,61.000000,210.504669,74.771332,138.126430,4.040580,4.928169,9.036307,16.073591,2.777178,40.026249,4.894222
52,1.0,53,CO,1995,1.524,3738061.0,82.582924,92946544,44.000000,167.350006,44.000000,109.809720,4.413803,4.698749,0.000000,16.315557,2.792119,28.871392,4.616546
53,1.0,54,CT,1995,1.524,3265293.0,79.472191,104315120,74.000000,218.280502,86.355499,143.228683,4.375407,4.964443,8.107283,20.962359,3.042728,48.556433,4.865674
54,1.0,55,DE,1995,1.524,718265.0,124.466599,18237436,48.000000,165.600006,48.000000,108.661426,4.824037,4.688237,0.000000,16.660734,2.813055,31.496064,4.616546
55,1.0,56,FL,1995,1.524,14185403.0,93.074554,333525344,57.900002,187.717178,68.525505,123.174007,4.533401,4.813598,6.972116,15.427737,2.736167,37.992129,4.830791
56,1.0,57,GA,1995,1.524,7188538.0,97.474625,159800448,36.000000,156.573074,37.431416,102.738243,4.579592,4.632184,0.939249,14.586546,2.680100,23.622048,4.645408
57,1.0,58,IA,1995,1.524,2840860.0,92.401596,60170928,60.000000,190.889999,69.089996,125.255911,4.526144,4.830359,5.964565,13.897988,2.631744,39.370081,4.799831


In [9]:
cig_ivreg = IV2SLS(c1995.lnpacks, exog=c1995.const, endog=c1995.lnrprice, instruments=c1995.salestax).fit()
print(cig_ivreg)




                          IV-2SLS Estimation Summary                          
Dep. Variable:                lnpacks   R-squared:                      0.4011
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3881
No. Observations:                  48   F-statistic:                    12.046
Date:                Fri, Oct 28 2022   P-value (F-stat)                0.0005
Time:                        13:22:23   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          9.7199     1.4961     6.4966     0.0000      6.7875      12.652
lnrprice      -1.0836     0.3122    -3.4708     0.00

In [10]:

cig_ivreg2 = IV2SLS(c1995.lnpacks, exog=c1995.const, endog=c1995[['lnrprice','lnrincome']], instruments=c1995[['salestax','lnrincome']]).fit()
print(cig_ivreg2)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                lnpacks   R-squared:                      0.4189
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3931
No. Observations:                  48   F-statistic:                    17.474
Date:                Fri, Oct 28 2022   P-value (F-stat)                0.0002
Time:                        13:22:23   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          9.4307     1.2194     7.7338     0.0000      7.0407      11.821
lnrprice      -1.1434     0.3605    -3.1718     0.00

In [11]:
cig_ivreg3 = IV2SLS(c1995.lnpacks, exog=c1995.const, endog=c1995[['lnrprice','lnrincome']], instruments=c1995[['salestax','lnrincome','cigtax']]).fit()
print(cig_ivreg3)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                lnpacks   R-squared:                      0.4294
Estimator:                    IV-2SLS   Adj. R-squared:                 0.4041
No. Observations:                  48   F-statistic:                    34.506
Date:                Fri, Oct 28 2022   P-value (F-stat)                0.0000
Time:                        13:22:23   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          9.8950     0.9288     10.654     0.0000      8.0746      11.715
lnrprice      -1.2774     0.2417    -5.2855     0.00

In [12]:
mod_relevance = mt.reg(c1995, 'lnrprice', ['salestax','cigtax','lnrincome',], addcons=True,vce_type="robust")
print(mod_relevance)

Dependent variable:	lnrprice
N:			48
R-squared:		0.9403
Estimation method:	OLS
VCE method:		Robust
          coeff    se      t   p>t CI_low CI_high
salestax  0.011 0.002  5.097 0.000  0.007   0.015
cigtax    0.009 0.001 10.752 0.000  0.008   0.011
lnrincome 0.108 0.040  2.732 0.009  0.028   0.188
_cons     4.103 0.088 46.425 0.000  3.925   4.281



In [13]:
from scipy.stats import ttest_ind

mod_relevance.Ftest(['salestax', 'cigtax'])

(209.67626939646124, 1.1102230246251565e-16)

In [18]:
c1995['resid'] = cig_ivreg3.resids
c1995.head(10)

cig_iv_OR = mt.reg(c1995, 'resid', ['salestax','cigtax','lnrincome'], addcons=True,vce_type="robust")

cig_iv_OR.Ftest(['salestax', 'cigtax'])

In [52]:
#Test de Hausman
import statsmodels.formula.api as smf

mod_relevance = mt.reg(c1995, 'lnrprice', ['salestax','cigtax','lnrincome'], addcons=True,vce_type="robust")
mod_relevance = smf.ols("lnrprice ~ salestax + cigtax + lnrincome", data=c1995).fit()
print(mod_relevance)
c1995['resid_mod_rel'] = mod_relevance.resid
c1995.head(10)

mod_test_HW = mt.reg(c1995, 'lnpacks', ['lnrprice','resid_mod_rel','lnrincome'], addcons=True,vce_type="robust")

Dependent variable:	lnpacks
N:			48
R-squared:		0.4697
Estimation method:	OLS
VCE method:		Robust
               coeff    se      t   p>t CI_low CI_high
lnrprice      -1.277 0.228 -5.601 0.000 -1.737  -0.818
resid_mod_rel -1.565 0.836 -1.872 0.068 -3.250   0.120
lnrincome      0.280 0.247  1.134 0.263 -0.218   0.779
_cons          9.895 0.866 11.432 0.000  8.151  11.639